# Simple Table Example Client
This Jupyter Notebook is a client for the simple table example.  Once you have started the Simple Table Example server, start this Notebook and run it to query the tables on the example server.

_The Simple Data Transfer Protocol requires *no* specialized software on the client!_ All that's needed is _any_ library to make http/s requests.  In this example, we use the standard Python library _requests_ but any example will do

In [1]:
import requests

Set the URL of the server to your local installation.  _This is the only variable you will need to adjust for your local installation_.

In [2]:
server_url = 'http://localhost:5001'

The '/' and '/help' routes show the available routes

In [3]:
response = requests.get(server_url)

Check the status of the response.  It should be 200 (everything is OK).  If it hangs on returns a 404 (Not found) then the server_url is wrong or blocked due to a local security setting.

In [4]:
response.status_code

200

All returns in the Simple Data Transfer Protocol are JSON, so these can be used in computation on the client.  Implementers are free to add HTML routes for clarity and UI, but this is not a requirement.  Help for this server generates a list of routes, required parameters, and their return values

In [5]:
response.json()

[{'description': 'Return the list of table names',
  'headers': 'None',
  'method': 'GET',
  'url': '/get_table_names'},
 {'description': 'Returns the schema of the table as a list of objects.  Each object will  contain the fields "name" and "type", where "type"is an SDML type.',
  'headers': 'None',
  'method': 'GET',
  'url': '/get_table_schema?table_name string, required'},
 {'description': 'Dumps a JSONIfied dictionary of the form:{table_name: <table_schema>}, where <table_schema> is a dictionary{"name": name, "type": type}',
  'headers': 'None',
  'method': 'GET',
  'url': '/get_tables'},
 {'body': {'columns': ' If  present, a list of the names of the columns to fetch',
   'filter': ' optional, a filter_spec in the SDTP filter language',
   'table': ' required, the name of the table to get the rows from'},
  'description': 'Get the rows from table table which match filter filter.  If columns is present, return only those columns.  Returns a simple list of lists of columns',
  'met

Get the list of table names

In [6]:
response = requests.get(f'{server_url}/get_table_names')
response.json()

['ec_table',
 'electoral_college',
 'nationwide_vote',
 'nightingale',
 'presidential_margins',
 'presidential_vote',
 'presidential_vote_history']

Get the schema of the nationwide_vote table

In [7]:
response = requests.get(f'{server_url}/get_table_schema?table_name=nationwide_vote')
response.json()

[{'name': 'Year', 'type': 'number'},
 {'name': 'Party', 'type': 'string'},
 {'name': 'Percentage', 'type': 'number'}]

Find the minimum and maximum years for which we have nationwide vote data

In [8]:
response = requests.get(f'{server_url}/get_range_spec?table_name=nationwide_vote&column_name=Year')
response.json()

[1828.0, 2020.0]

Find all the parties in the nationwide vote column

In [9]:
response = requests.get(f'{server_url}/get_all_values?table_name=nationwide_vote&column_name=Party')
response.json()

['American',
 'American Independent',
 'Anti-Masonic',
 'Constitutional Union',
 'Democratic',
 'Free Soil',
 'Green',
 'Greenback',
 'Independent',
 'Libertarian',
 'Liberty',
 'Other',
 'Populist',
 'Progressive',
 'Reform',
 'Republican',
 'Socialist',
 'Southern Democratic',
 'States\' Rights "Dixiecrat"']

Get the schema of the presidential_vote table

In [10]:
response = requests.get(f'{server_url}/get_table_schema?table_name=presidential_vote')
response.json()

[{'name': 'Year', 'type': 'number'},
 {'name': 'State', 'type': 'string'},
 {'name': 'Name', 'type': 'string'},
 {'name': 'Party', 'type': 'string'},
 {'name': 'Votes', 'type': 'number'},
 {'name': 'Percentage', 'type': 'number'}]

Find all the years in which a 'Roosevelt' ran for President

In [11]:
filter_name = {"operator": "REGEX_MATCH", "column": "Name", "expression": ".*Roosevelt.*"}
filter_state = {"operator": "IN_LIST", "column": "State", "values": ["Nationwide"]}
all_filter = {"operator": "ALL", "arguments": [filter_name, filter_state]}
query = {"table": "presidential_vote",  "filter": all_filter, "columns": ['Year', 'Name', 'Percentage']}
response = requests.post(f'{server_url}/get_filtered_rows', json = query)
response.status_code

200

In [12]:
response.text

'[[1944.0,"Roosevelt, Franklin (FDR)",53.3857200015374],[1940.0,"Roosevelt, Franklin (FDR)",54.72428023147225],[1936.0,"Roosevelt, Franklin (FDR)",60.799043018603165],[1932.0,"Roosevelt, Franklin (FDR)",57.40906712325231],[1912.0,"Roosevelt, Theodore \\"Teddy\\"",27.387567058051598],[1904.0,"Roosevelt, Theodore (Teddy)",56.41777007850961]]\n'